In [114]:
import spotipy
import json
client_id = '45ef4a3faa564338bb59b7b84d5ec2e9'
client_secret = 'f0f192735900439caffb9ee27d6345a1'
redirect_uri='http://localhost:8889/callback'
scope = 'user-read-currently-playing'

token = spotipy.oauth2.SpotifyPKCE(client_id=client_id,redirect_uri=redirect_uri,scope=scope).get_access_token()
spotify = spotipy.client.Spotify(auth=token)
data_about_song = spotify.current_user_playing_track()

Downloads currently playing song to myfile.mp3. This can be changed

In [115]:
import requests
songfile_destination = 'myfile.mp3'
url = data_about_song['item']['preview_url']
doc = requests.get(url)
with open(songfile_destination, 'wb') as f:
        f.write(doc.content)

In [116]:
from pydub import AudioSegment

# files                                                                         
src = "vocal.wav"
dst = "doff.wav"

# convert wav to mp3                                                            
sound = AudioSegment.from_mp3(src)
sound.export(dst, format="wav")

<_io.BufferedRandom name='doff.wav'>

In [107]:
import os
import json
import numpy as np
import librosa
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
import sys


# Sampling rate.
sr = 22050

# Let's make sure all files have the same amount of samples and pick a duration right under 30 seconds.
TOTAL_SAMPLES = 29 * sr

# The dataset contains 999 files. Lets make it bigger. 
# X amount of slices => X times more training examples.
NUM_SLICES = 10
SAMPLES_PER_SLICE = int(TOTAL_SAMPLES / NUM_SLICES)

NUM_SLICES = 10
SAMPLES_PER_SLICE = int(TOTAL_SAMPLES / NUM_SLICES)

def prepare_datasets(inputs, targets, split_size):
    
    # Creating a validation set and a test set.
    inputs_train, inputs_val, targets_train, targets_val = train_test_split(inputs, targets, test_size=split_size)
    inputs_train, inputs_test, targets_train, targets_test = train_test_split(inputs_train, targets_train, test_size=split_size)
    
    # Our CNN model expects 3D input shape.
    inputs_train = inputs_train[..., np.newaxis]
    inputs_val = inputs_val[..., np.newaxis]
    inputs_test = inputs_test[..., np.newaxis]
    
    return inputs_train, inputs_val, inputs_test, targets_train, targets_val, targets_test

def load_data(json_path):

    with open(json_path, 'r') as f:
        data = json.load(f)
        #data = unicode(data, errors='replace')
    f.close()

    # Let's load our data into numpy arrays for TensorFlow compatibility.
    X = np.array(data["mfcc"])
    y = np.array(data["labels"])

    return X, y

def preprocess_data(source_path, json_path):

    # Let's create a dictionary of labels and processed data.
    mydict = {
        "labels": [],
        "mfcc": []
        }

    # Let's browse each file, slice it and generate the 13 band mfcc for each slice.
    for i, (dirpath, dirnames, filenames) in enumerate(os.walk(source_path)):
        for file in filenames:
            # exclude a corrupted wav file that makes everything crash.
            if os.path.join(dirpath, file) != 'data\genres_original\jazz\jazz.00054.wav':
                song, sr = librosa.load(os.path.join(dirpath, file), duration=29)
                for s in range(NUM_SLICES):
                    start_sample = SAMPLES_PER_SLICE * s
                    end_sample = start_sample + SAMPLES_PER_SLICE
                    mfcc = librosa.feature.mfcc(y=song[start_sample:end_sample], sr=sr, n_mfcc=13)
                    mfcc = mfcc.T
                    mydict["labels"].append(i-1)
                    mydict["mfcc"].append(mfcc.tolist())
            else:
                pass

    # Let's write the dictionary in a json file.    
    with open(json_path, 'w') as f:
        json.dump(mydict, f)
    f.close()


model = tf.keras.models.load_model('mymodel')

def tai_prediction(model, X, y, idx):
    
    genre_dict = {
        0 : "blues",
        1 : "classical",
        2 : "country",
        3 : "disco",
        4 : "hiphop",
        5 : "jazz",
        6 : "metal",
        7 : "pop",
        8 : "reggae",
        9 : "rock",
        }
        
    predictions = model.predict(X)
    genre = np.argmax(predictions[idx])
    
    #print("\n---Now testing the model for one audio file---\nThe model predicts: {}, and ground truth is: {}.\n".format(genre_dict[genre], genre_dict[y[idx]]))
    print("\n---Now testing the model for one audio file---\nThe model predicts: {}, but what was it actually? \n".format(genre_dict[genre]))

def super_prediction(path, target):
    preprocess_data(source_path=path,json_path=target)
    inputs, targets = load_data(json_path=target)

    Xtrain, Xval, Xtest, ytrain, yval, ytest = prepare_datasets(inputs, targets, 0.2)
    tai_prediction(model,Xtest,ytest,1)

music_file = 'soundfile'#sys.argv[0]
music_target = 'data.json'#sys.argv[1]

super_prediction(music_file, music_target)

#inputs, targets = load_data(json_path=music_target)
#stuff = train_test_split(inputs, targets, test_size=0.2)
#model.predict(stuff[2])

1/1 [==============================] - 0s 169ms/step

---Now testing the model for one audio file---
The model predicts: classical, but what was it actually? 



In [119]:
bro = spotify.recommendations(seed_genres={'classical'})

In [141]:
song_list = []
uri_list = []
for i in range(len(bro['tracks'])):
    song_list.append(bro['tracks'][i]['name'])
    uri_list.append(bro['tracks'][i]['uri'])

In [143]:
spotify.add_to_queue(uri_list[0])

HTTP Error for POST to https://api.spotify.com/v1/me/player/queue?uri=spotify:track:2mYuazFCuAeDsU2EkY68Bp with Params: {} returned 403 due to Player command failed: Premium required


SpotifyException: http status: 403, code:-1 - https://api.spotify.com/v1/me/player/queue?uri=spotify:track:2mYuazFCuAeDsU2EkY68Bp:
 Player command failed: Premium required, reason: PREMIUM_REQUIRED